In [1]:
from database import *
from backtest import *
import json

/opt/anaconda3/lib/python3.8/site-packages/shioaji/__init__.py:20: Warning: Mac OS currently don't provide ca module all function about placing order is not working.
  warnings.warn("Mac OS currently don't provide ca module all function about placing order is not working.", Warning)


In [2]:
with open('config.json') as f:
  config = json.load(f)
print(config)
api = sj.Shioaji()
api.login(
    person_id=config['account'], 
    passwd=config['password'], 
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
)

connection = sqlite3.connect('data.db')

# api.activate_ca(
#     ca_path="./Sinopac.pfx",
#     ca_passwd="YOUR_CA_PASSWORD",
#     person_id="Person of this Ca",
# )

{'account': 'O100318574', 'password': 'YHHung0903'}
Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.


In [3]:
ticks = {}
kbars = {}
volume_today = {}

open_orders = {}
order_qty = {}
positions = {}

workers = {}
stop_threads = False

@api.quote.on_quote
def quote_callback(topic: str, quote: dict):
    
    code = topic.split('/')[-1]
    
    ts = pd.to_datetime(quote.get('Date') + ' ' + quote.get('Time'))
    
    if ts.hour < 9:
        return 
    
    close = quote.get('Close')[0]
    volume = quote.get('Volume')[0]
    vol_sum = quote.get('VolSum')[0]
    
    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum
    
    
def place_cb(stat, msg):
    
    if stat.value == 'TFTDEAL':

        code = msg['code']
        qty = int(msg['quantity'])

        order_qty[code] -= qty
        
        if order_qty[code] == 0:
            open_orders[code] = None
        
        position = positions.get(code, 0)
        
        if msg['action'] == 'Sell':
            qty = qty * -1
            
        position += qty
        
        positions[code] = position 

api.set_order_callback(place_cb)

In [4]:
import time

def tbot(code):

    print('=== 啟動 tbot #{} ==='.format(code))
    
    tw_calendar = get_calendar('XTAI')
    
    date = pd.to_datetime(datetime.datetime.now().date())
    prev_trading_date = tw_calendar.previous_close(date).date()
    
    if date not in tw_calendar.opens:
        print('今日非交易日')
        return 

    ticks[code] = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
    ticks[code] = ticks[code][['close', 'volume']]
    kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='5Min'))
    kbars[code] = kbars[code][date:]
    volume_today[code] = kbars[code]['volume'].sum()
    entry_price = -1
    open_price = -1
    api.quote.subscribe(api.Contracts.Stocks[code], quote_type='tick')
    
    while True:
        
        if stop_threads:
            break
            
        time.sleep(1)
        
        current_time = datetime.datetime.now()
        
        if current_time.second == 0:

            kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='5Min'))
            kbars[code] = kbars[code][date:]
            
            str_current_time = (current_time - datetime.timedelta(minutes=5)).strftime('%Y-%m-%d %H:%M:%S')
            str_open_time = current_time.replace(hour=9, minute=0, second=0).strftime('%Y-%m-%d %H:%M:%S')
            try:
                current_price = kbars[code].loc[str_current_time]['close']
                if open_price == -1:
                    open_price = kbars[code].loc[str_open_time]['open']
                    # 開盤太高，不沖
                    if open_price * 1.04 <= kbars[code].loc[str_open_time]['close']:
                        break
            except:
                continue
 
            open_order = open_orders.get(code, None)
            
            if open_order:
                
                api.update_status(api.stock_account)
                
                if open_order.status.status in ['PendingSubmit', 'Submitted', 'Filling']:
                    if current_time > current_time.replace(hour=10, minute=30, second=0) and open_order.order.action == 'Buy':
                        api.cancel_order(open_order)
                        open_orders[code] = None
                        order_qty[code] = 0
                        if open_order.status.status != 'Filling':
                            break
                    elif current_time >= current_time.replace(hour=13, minute=20, second=0) and open_order.order.action == 'Sell':
                        api.cancel_order(open_order)
                        open_orders[code] = None
                        order_qty[code] = 0
#                         if open_order.status.status != 'Filling':
#                             break                                        
            position = positions.get(code, 0)
            
            if (
                current_time <= current_time.replace(hour=10, minute=30, second=0) and
                kbars[code].loc[str_current_time]['cci'] < -100 and
                kbars[code].loc[str_current_time]['slowk'] < 30 and
                kbars[code].loc[str_current_time]['rsi'] < 30 and
                position == 0
            ):
                if current_price < 20:
                    qty = 20
                else:
                    qty = 2
                    
                contract = api.Contracts.Stocks[code]
                order = api.Order(
                    price=current_price,
                    quantity=qty,
                    action="Buy",
                    price_type="LMT",
                    order_type="ROD",
                    account=api.stock_account
                )
                    
                trade = api.place_order(contract, order)

                open_orders[code] = trade
                order_qty[code] = qty

                entry_price = current_price
                print('[{}] buy {} at {}'.format(str_current_time, code, current_price))

                
            elif (
                (current_price >= entry_price * 1.03 and position != 0) or
                (current_price <= entry_price * 0.98 and position != 0) or
                (current_price > entry_price and kbars[code].loc[str_current_time]['slowk'] > 70 and position != 0) or
                (current_price > entry_price and kbars[code].loc[str_current_time]['rsi'] > 70 and position != 0) or
                (current_price >= entry_price * 1.01 and kbars[code].loc[str_current_time]['cci'] < -60 and position != 0) or 
                (current_price > entry_price and kbars[code].loc[str_current_time]['cci'] > 200 and position != 0) or
                (current_price > entry_price and kbars[code].loc[str_current_time]['upper'] < current_price and position != 0)
            ):
                        
                contract = api.Contracts.Stocks[code]
                order = api.Order(
                    price=current_price,
                    quantity=position,
                    action="Sell",
                    price_type="LMT",
                    order_type="ROD",
                    account=api.stock_account
                )
                        
                trade = api.place_order(contract, order)

                open_orders[code] = trade
                order_qty[code] = position
                        
                print('[{}] sell {} at {}'.format(str_current_time, code, current_price))
                break
                    
            elif (
                current_time >= current_time.replace(hour=13, minute=20, second=0)
            ):
                if position != 0:
                    
                    contract = api.Contracts.Stocks[code]
                    order = api.Order(
                        price=0,
                        quantity=position,
                        action="Sell",
                        price_type="MKT",
                        order_type="ROD",
                        account=api.stock_account
                    )

                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = position

                    print('[{}] sell {} at {}'.format(str_current_time, code, current_price))
                    
                break 
                
    api.quote.unsubscribe(api.Contracts.Stocks[code], quote_type='tick')
    
    print('=== 關閉 tbot #{} ==='.format(code))


In [5]:
import threading

def run():
    
    date = pd.to_datetime(datetime.datetime.now().date())
    
    codes = get_stocks(date, connection)    
    day_trading_codes = [code for code in codes if get_stock(code, connection, api)[0].iloc[0]['day_trade'] == 'Yes']

    print('今日當沖標的：{}'.format(day_trading_codes))
    
    for code in day_trading_codes:
        
        worker = threading.Thread(name=code, target=tbot, args=(code,))
        worker.start()
        workers[code] = worker

In [6]:
update_historial_data('2021/4/26', '2021/4/26', connection, api)

<SecurityType.Future: 'FUT'> fetch done.
正在更新每日收盤行情...
正在篩選每日當沖標的...
[datetime.date(2021, 4, 20) datetime.date(2021, 4, 21)
 datetime.date(2021, 4, 22) datetime.date(2021, 4, 23)]
<SecurityType.Stock: 'STK'> fetch done.
<SecurityType.Option: 'OPT'> fetch done.
正在更新股票資訊...
正在更新逐筆成交資料...
正在更新2021/04/23逐筆成交資料，總共4檔標的
股市資料庫更新完成


In [7]:
run()

[datetime.date(2021, 4, 20) datetime.date(2021, 4, 21)
 datetime.date(2021, 4, 22) datetime.date(2021, 4, 23)]
今日當沖標的：['5471', '6226', '2376', '3189']
=== 啟動 tbot #5471 ===
今日非交易日
=== 啟動 tbot #6226 ===
今日非交易日
=== 啟動 tbot #2376 ===
今日非交易日
=== 啟動 tbot #3189 ===
今日非交易日


In [8]:
workers

{'5471': <Thread(5471, stopped 123145437085696)>,
 '6226': <Thread(6226, stopped 123145437085696)>,
 '2376': <Thread(2376, stopped 123145437085696)>,
 '3189': <Thread(3189, stopped 123145437085696)>}

In [9]:
ticks
kbars
volume_today

{}

In [10]:
stop_threads = True